In [49]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import json
import re
import sqlite3


In [59]:
df_CEPsetor = pd.read_csv('dados/dados_CEPvsSETOR.csv') #Na cidade de São Paulo
df_CEPsetor.head(5)

,CD_SETOR,CEP
0,355030801000001,03190-130
1,355030801000001,03191-090
2,355030801000001,03191-100
3,355030801000001,03191-110
4,355030801000001,03191-120


In [65]:
df_CEPs_por_setor = df_CEPsetor.groupby('CD_SETOR').agg({'CEP': lambda x: list(x)}).reset_index()
df_CEPs_por_setor.head()

,CD_SETOR,CEP
0,355030801000001,"[03190-130, 03191-090, 03191-100, 03191-110, 0..."
1,355030801000002,"[03345-045, 03349-040, 03349-045, 03350-020, 0..."
2,355030801000003,[03350-060]
3,355030801000004,"[03345-001, 03350-010, 03350-015, 03351-120]"
4,355030801000005,"[03191-160, 03192-140, 03192-170, 03345-050]"


In [45]:

CEP = '04119010'
def scraper(CEP):
    driver = webdriver.Chrome()

    driver.get('https://www.quintoandar.com.br/comprar/imovel/'+CEP)
    time.sleep(5)

    for i in range(9):
        try:
            ver_mais_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Ver mais')]")
            ver_mais_button.click()  # Clica no botão
            time.sleep(3)  # Aguarde o conteúdo carregar após o clique
        except:
            pass

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    soup = soup.find('html')
    soup = soup.prettify()

    return soup

In [46]:
scraper(CEP)

'<html lang="pt-BR">\n <head>\n  <style>\n   .LGLeeN-keyboard-shortcuts-view{display:-webkit-box;display:-webkit-flex;display:-moz-box;display:-ms-flexbox;display:flex}.LGLeeN-keyboard-shortcuts-view table,.LGLeeN-keyboard-shortcuts-view tbody,.LGLeeN-keyboard-shortcuts-view td,.LGLeeN-keyboard-shortcuts-view tr{background:inherit;border:none;margin:0;padding:0}.LGLeeN-keyboard-shortcuts-view table{display:table}.LGLeeN-keyboard-shortcuts-view tr{display:table-row}.LGLeeN-keyboard-shortcuts-view td{-moz-box-sizing:border-box;box-sizing:border-box;display:table-cell;color:#000;padding:6px;vertical-align:middle;white-space:nowrap}.LGLeeN-keyboard-shortcuts-view td:first-child{text-align:end}.LGLeeN-keyboard-shortcuts-view td kbd{background-color:#e8eaed;border-radius:2px;border:none;-moz-box-sizing:border-box;box-sizing:border-box;color:inherit;display:inline-block;font-family:Google Sans Text,Roboto,Arial,sans-serif;line-height:16px;margin:0 2px;min-height:20px;min-width:20px;padding:2p

In [47]:
def extract_data(cep):
    dados = getDadosCEP(cep)
    bairro = dados['bairro']
    cidade = dados['localidade']
    estado = dados['uf']
    regiao = dados['regiao']
    bairro = bairro.replace(" ", "-").lower()
    soup = scraper(bairro)
    soup = BeautifulSoup(soup, 'html.parser')
    casas = soup.find_all(attrs={"data-testid": "house-card-container"})
    for casa in casas:
        preco = find_preco_imovel(casa)
        m2 = find_m2_imovel(casa)
        rua = find_rua_imovel(casa)
        preco_m2 = float(preco) / float(m2)
        
        # Check if the data already exists
        cursor.execute('''
        SELECT id FROM data 
        WHERE cep = ? AND bairro = ? AND preco = ? AND m2 = ?
        ''', (cep, bairro, preco, m2))
        
        if cursor.fetchone() is None:
            # Data doesn't exist, so insert it
            add_data(cep, rua, bairro, cidade, estado, regiao, preco, m2, preco_m2)
